In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df1=pd.read_pickle('cleaned_travel.pkl')

In [3]:
#model training
X_train=pd.read_pickle("X_train.pkl")
X_test=pd.read_pickle("X_test.pkl")
y_train=pd.read_pickle("y_train.pkl")
y_test=pd.read_pickle("y_test.pkl")

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,accuracy_score,classification_report,roc_auc_score,roc_curve,confusion_matrix,f1_score,precision_score,recall_score

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

In [8]:
models={
    'GradientBoostClassifier':GradientBoostingClassifier(),
    'GradientBoostRegressor':GradientBoostingRegressor(),
}

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    y_train_prob=model.predict_proba(X_train)[:, 1]
    y_test_prob=model.predict_proba(X_test)[:, 1]

    # Training metrics
    model_train_mae=mean_absolute_error(y_train,y_train_pred)
    model_train_rmse=np.sqrt(mean_squared_error(y_train,y_train_pred))
    model_train_r2=r2_score(y_train,y_train_pred)
    model_train_accuracy  = accuracy_score(y_train, y_train_pred)
    model_train_precision = precision_score(y_train, y_train_pred)
    model_train_recall    = recall_score(y_train, y_train_pred)
    model_train_f1        = f1_score(y_train, y_train_pred)
    model_train_roc_auc   = roc_auc_score(y_train, y_train_prob)
    model_train_cm        = confusion_matrix(y_train, y_train_pred)

    # Testing metrics
    model_test_mae=mean_absolute_error(y_test,y_test_pred)
    model_test_rmse=np.sqrt(mean_squared_error(y_test,y_test_pred))
    model_test_r2=r2_score(y_test,y_test_pred)
    model_test_accuracy  = accuracy_score(y_test, y_test_pred)
    model_test_precision = precision_score(y_test, y_test_pred)
    model_test_recall    = recall_score(y_test, y_test_pred)
    model_test_f1        = f1_score(y_test, y_test_pred)
    model_test_roc_auc   = roc_auc_score(y_test, y_test_prob)
    model_test_cm        = confusion_matrix(y_test, y_test_pred)

    print(list(models.keys())[i])

    #printing out the results ith model names for training set
    print("----- TRAINING METRICS -----")
    print(f"model_train_accuracy:   {model_train_accuracy:.4f}")
    print(f"model_train_precision:  {model_train_precision:.4f}")
    print(f"model_train_recall:     {model_train_recall:.4f}")
    print(f"model_train_f1:         {model_train_f1:.4f}")
    print(f"model_train_roc_auc:    {model_train_roc_auc:.4f}")
    print(f"model_train_mae:        {model_train_mae:.4f}")
    print(f"model_train_rmse:       {model_train_rmse:.4f}")
    print(f"model_train_r2:         {model_train_r2:.4f}")
    print("model_train_cm:\n", model_train_cm)

    #printing out the results ith model names for tested set
    print("\n----- TESTING METRICS -----")
    print(f"model_test_accuracy:   {model_test_accuracy:.4f}")
    print(f"model_test_precision:  {model_test_precision:.4f}")
    print(f"model_test_recall:     {model_test_recall:.4f}")
    print(f"model_test_f1:         {model_test_f1:.4f}")
    print(f"model_test_roc_auc:    {model_test_roc_auc:.4f}")
    print(f"model_test_mae:        {model_test_mae:.4f}")
    print(f"model_test_rmse:       {model_test_rmse:.4f}")
    print(f"model_test_r2:         {model_test_r2:.4f}")
    print("model_test_cm:\n", model_test_cm)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [9]:
GBR_params={
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [2, 3, 4],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "subsample": [1.0, 0.8, 0.6],
    "max_features": ["auto", "sqrt", "log2"]
}

GBC_params={
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [2, 3, 4],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "subsample": [1.0, 0.8, 0.6],
    "max_features": ["auto", "sqrt", "log2"]
}

In [10]:
#RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
randomcv_models=[
    ('GBR',GradientBoostingRegressor(),GBR_params),
    ('GBC',GradientBoostingClassifier(),GBC_params)
]
model_params={}

for name,model,params in randomcv_models:
    random=RandomizedSearchCV(
        estimator=model,
        n_jobs=-1,
        cv=3,
        n_iter=10,
        param_distributions=params
    )
    random.fit(X_train,y_train)
    model_params[name]=random.best_params_

for model_name in model_params:
    print(f'-----Best params for {model_name}-----')
    print(model_params[model_name])

/home/ahmed0001/Machine-Learning/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:516: FitFailedWarning: 
6 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ahmed0001/Machine-Learning/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ahmed0001/Machine-Learning/.venv/lib/python3.12/site-packages/sklearn/base.py", line 1358, in wrapper
    estimator._validate_params()
  File "/home/ahmed0001/Machine-Learning/.venv/lib/python3.12/site-packages/sklearn/base.py", line 471, in _valid

KeyboardInterrupt: 

# Model is taking too much time tpo work so basically use the best params and then execute the model again

### Same for XGBOOST